In [1]:
import numpy as np
import sympy as sp
np.set_printoptions(precision = 2, suppress = True) 

$\text{Task 1}.$

Solve the system of equations approximately and assess the solution's accuracy using $L_{1}, L_{2}, L_{\infty}$ norms.

\begin{equation}
    \begin{cases}
    (4 + \epsilon_{1})x + 4(-2 + \epsilon_2)y = 4 + \epsilon_3 \\
    -2x + (1 + \epsilon_1)y = 1 + \epsilon_4
    \end{cases}\,.
\end{equation}

... ->

\begin{equation}
    \begin{cases}
    (4 + \epsilon_{1})x + (-8 + 4\epsilon_2)y = 4 + \epsilon_3 \\
    -2x + (1 + \epsilon_1)y = 1 + \epsilon_4
    \end{cases}\,.
\end{equation}



In [2]:
A = np.array([[4, -8], [-2, 1]])
b = np.array([4, 1])

solution = np.linalg.inv(A) @ b
solution

array([-1., -1.])

Yielding an exact solution by calculating $A^{-1}b$ $\implies e_i = 0 \forall i$. That means the error estimation for $L_{1}, L_{2}$ and $L_{\infty}$ = 0.

**Answer (Task 1)**: error for $L_{1}, L_{2}$ and $L_{\infty}$ = 0. An exact solution can be obtained.

___
$\text{Task 2}.$ 

Solve the system of equations using the iteration method.

\begin{equation}
    \begin{cases}
    21x + 1y + 4z = 6 \\
    1x + 21y + 9z = 3 \\
    1x + 5y + 22z = 4 
    \end{cases}
\end{equation}

...

\begin{equation}
    \begin{cases}
    x = \frac{6}{21} - \frac{1}{21}y - \frac{4}{21}z \\
    y = \frac{3}{21} - \frac{1}{21}x - \frac{9}{21}z \\
    z = \frac{4}{22} - \frac{1}{22}x - \frac{5}{22}y  
    \end{cases}
\end{equation}

Derive the matrix $\mathbf{P}$. 

$$\mathbf{P} = \begin{bmatrix}
0 & -\frac{1}{21} & - \frac{4}{21} \\
- \frac{1}{21} & 0 & - \frac{9}{21} \\
- \frac{1}{22} & - \frac{5}{22} & 0
\end{bmatrix}$$

In [3]:
A = np.array([[21, 1, 4], [1, 21, 9], [1, 5, 22]])
b = np.array([6, 3, 4])

P = np.array([
  [0, -1/21, -4/21],
  [-1/21, 0, -9/21],
  [-1/22, -5/22, 0]
])
P

array([[ 0.  , -0.05, -0.19],
       [-0.05,  0.  , -0.43],
       [-0.05, -0.23,  0.  ]])

In [4]:
P_norm = np.linalg.norm(P, ord = np.inf)
"""
Since the L_inf-norm of matrix P is less than 1, 
the iteration method is going to converge and the solution will eventually be obtained.
"""
P_norm

0.47619047619047616

In [5]:
# Iteration method.

MAX_ITERATIONS = 100
PRECISION_THRESHOLD = 0.001

x = [0, 0, 0]
coefficients = [6/21, 3/21, 4/22]

for iteration in np.arange(1, MAX_ITERATIONS, 1):
    xn = coefficients + (P @ x)
    diff = x - xn
    x = xn
    if np.all(np.abs(diff) < PRECISION_THRESHOLD):
        break

print(iteration)             # Number of the iteration where algorithm converges.
print(xn)                    # Solution obtained using iteration method

7
[0.25 0.06 0.16]


In [6]:
# Double check!
print(A @ xn)                # Verify (Ax ~ b)
print(np.linalg.solve(A, b)) # Solution by numpy.

[6. 3. 4.]
[0.25 0.06 0.16]


In [7]:
error = (P_norm ** (iteration) / (1 - P_norm)) * np.linalg.norm(coefficients, ord = 1)
error

0.006469916551417038

**Answer (Task 2)** 

Iteration: $7$, error: $\frac{ ||P||_1^{7} }{1-||P||_1}|c| \approx 0.006$.

Solution: $[0.25, 0.06, 0.16]$.

___
$\text{Task 3}.$ Find the eigenvalues for the following matrix using Jacobi eigenvalue method:

$$\mathbf{A} = \begin{bmatrix}
3 & 2 & -6 \\
2 & 8 & 6 \\
-6 & 6 & -8
\end{bmatrix}$$

The iteration of the algorithm can be described as follows:

1. First, apply the $\text{abs}$ to each element and find the maximal non-diagonal element in the matrix, e.g. $a_{32} = 6.$ Result is denoted as $a_{ij}$.

2. Then, calculate $\text{tg}(2\phi) = \frac{2a_{ij}}{a_{ii} - a_{jj}}$, obtain $\phi$ = $\text{arctg}(\frac{2a_{ij}}{a_{ii} - a_{jj}}) \space / \space 2$.
3. Construct the $R(\phi)$ rotation matrix, where $R_{ii} = R_{jj} = \cos\phi, R_{ij} = -\sin{\phi}, R_{ji} = \sin{\phi}$.
4. Calculate $A_{k + 1} = R^T A_k R$, repeat the iteration until the result is an almost diagonal matrix (in this case just diagonal will be sufficient).

In [8]:
MAX_ITERATIONS = 1000
A = np.array([[3, 2, -6], [2, 8, 6], [-6, 6, -8]])

def jacobi_eigenvalues(A, iterations = MAX_ITERATIONS):
    # Rotation routine for the Jacobi eigenvalue algorithm.
    def rotation_iteration(A):
        # Get the absolute value maximal non-diagonal element.
        def maximal_non_diagonal(A):
            A = np.abs(A)
            np.fill_diagonal(A, 0)
            i, j = np.unravel_index(np.argmax(A, axis = None), A.shape)
            return i, j
        i, j = maximal_non_diagonal(A)
        phi = np.arctan(2 * A[i][j] / (A[i][i] - A[j][j])) / 2
        R = np.eye(A.shape[0])
        R[i][i] = R[j][j] = np.cos(phi)
        R[i][j] = -np.sin(phi)
        R[j][i] = np.sin(phi)
        return R.T @ A @ R
    # Iteration process.
    for iteration in np.arange(MAX_ITERATIONS):
        A = rotation_iteration(A)
        is_diagonal = not np.count_nonzero(A - np.diag(np.diagonal(A)))
        if is_diagonal:
            break
    # Diagonal elements of resulting matrix should be our eigenvalues.
    return np.diag(A)

# Get the diagonal elements.
jacobi_eig = jacobi_eigenvalues(A)

# Check the solution.
eig, rvecs = np.linalg.eig(A)

print(jacobi_eig) 
print(eig)

[  5.57  10.   -12.57]
[  5.57 -12.57  10.  ]


**Answer (Task 3).** $\text{eig}(A) = (5.57, 10, -12.57).$

___
$\text{Task 4.}$ Find the eigenvalues for the matrix using QR-decomposition.

$$\mathbf{A} = \begin{bmatrix}
7 & 5 \\
5 & 2 \\
\end{bmatrix}$$

QR eigenvalues algorithm is defined as follows: $$
\space A_k = Q_kR_k \\
\space A_{k+1}=R_kQ_k
$$

Iterate until the matrix $A$ is diagonal.

In [9]:
A = np.array([[7, 5], [5, 2]])
MAX_ITERATIONS = 25

def qr_eigenvalues(A):
    for iteration in np.arange(MAX_ITERATIONS):
        Q, R = np.linalg.qr(A)
        A = R @ Q
        is_diagonal = not np.count_nonzero(A - np.diag(np.diagonal(A)))
        if is_diagonal:
            break
    return np.diag(A)
        
qr_eig = qr_eigenvalues(A)
eig, rvecs = np.linalg.eig(A) 

print(qr_eig) 
print(eig)

[10.09 -1.09]
[10.09 -1.09]


**Answer (Task 4):** $\text{eig}(A) = (10.09, -1.09)$.

___
$\text{Task 5.}$ Find the most influential node in the oriented graph using PageRank algorithm with the damping factor of $1 - \beta = 0.85$, and the adjacency matrix is defined as follows:
$$
A=
\begin{bmatrix}
    1 & 1 & 0 & 1 & 0\\
    1 & 0 & 0 & 1 & 1\\
    0 & 1 & 1 & 0 & 1\\
    1 & 1 & 0 & 0 & 0\\
    1 & 0 & 0 & 0 & 0\\
\end{bmatrix}
$$

Apply the iteration algorithm to find the most influential node:
$$
P^{m}_{\beta} \begin{bmatrix}
           \frac{1}{n} \\
           \frac{1}{n} \\
           \vdots \\
           \frac{1}{n}
         \end{bmatrix}
\forall m
$$

Matrix $P$ is $A^T$, in which every element is divided by sum of its column entries. Set the initial value of $x$ to $x = [\frac{1}{5} \cdots \frac{1}{5}]$.

Damping factor is defined as $$P_{\beta} = \frac{\beta}{n}1_{n\times1} + (1 - \beta)P$$

In [10]:
MAX_ITERATIONS = 100
PRECISION_THRESHOLD = 0.001

A = np.array(
[
    [1.0, 1.0, 0.0, 1.0, 0.0],
    [1.0, 0.0, 0.0, 1.0, 1.0],
    [0.0, 1.0, 1.0, 0.0, 1.0],
    [1.0, 1.0, 0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0, 0.0, 0.0]
])

def most_influential_node(A, beta, iterations = MAX_ITERATIONS):
    
    # Get the shape of the matrix.
    N = A.shape[0]
    
    # Find the matrix P -- divide the elements by sum of column entries.
    for i in np.arange(N):
        A[:, i] = A[:, i] / np.sum(A[:, i])
    
    # Initial value.
    x = np.ones(N) / N

    # Iterative algorithm with damping factor.
    for iteration in np.arange(iterations):
        xn = beta / N * np.ones_like(x) + (1 - beta) * (A @ x)
        diff = xn - x
        x = xn
        if np.all(np.abs(diff) < PRECISION_THRESHOLD):
            break
    return np.argmax(x), np.max(x), x
        
node, influential_score, scores = most_influential_node(A, 0.85)
node, influential_score, scores

(2, 0.22768359375, array([0.2 , 0.2 , 0.23, 0.19, 0.18]))

**Answer (Task 5).** The most influential node is node #2 ($\mathbf{C}$), $\text{Score} = 0.23$.
NB! Index starts from 0!

___
$\text{Task 6.}$ Find the Perron-Frobenius number and corresponding eigenvector with non-negative entries such that its euclidean norm is equal to 1. Is this matrix considered "productive"? Perron-Frobenius number must be found iteratively.

$$
A=
\begin{bmatrix}
    0.9  & 0.55 & 0.25 \\
    0.55 & 0.6  & 0.65 \\
    0.25 & 0.65 & 0.3  \\
\end{bmatrix}
$$

1. Find the eigenvalues iteratively (by using Jacobi eigenvalue method in Task 3).
2. Find the eigenvector by solving $(A - \lambda_{max}I)v = 0$ for $v$, then normalize $v$.

$$
(A - \lambda_{max}I)v =
\begin{bmatrix}
    -0.71  & 0.55 & 0.25 \\
    0.55 & -1.01  & 0.65 \\
    0.25 & 0.65 & -1.31  \\
\end{bmatrix}
v = 0 
\rightarrow v = [0.64, 0.63, 0.44] $$ 

3. Check two conditions for productivity: $\lambda_{max} < 1$ and $\left \| A \right \|_1 < 1$



In [11]:
A = np.array([[0.9, 0.55, 0.25], [0.55, 0.6, 0.65], [0.25, 0.65, 0.3]])
eigenvalues = jacobi_eigenvalues(A, iterations = 100)
print(f'Eigenvalues: {eigenvalues}')
print(A - np.max(eigenvalues) * np.eye(3)) # A - \lambda_max * E.

Eigenvalues: [ 0.43  1.61 -0.24]
[[-0.71  0.55  0.25]
 [ 0.55 -1.01  0.65]
 [ 0.25  0.65 -1.31]]


In [12]:
np.linalg.norm(np.array([0.64, 0.63, 0.44]), ord = 2)

1.0000499987500624

In [13]:
np.linalg.norm(np.array([0.64, 0.63, 0.44]), ord = 1)

1.71

**Answer (Task 6).** $\lambda_{max} = 1.61 > 1, \left \| A \right \|_1 \approx 1.71 > 1 \rightarrow \text{not productive}, v = [0.64, 0.63, 0.44], \lambda_{max} = 1.61$.


___
$\text{Task 7.}$ Find the vector of final demand. Construct a matrix of expenditure (technological) coefficients. Calculate the Leontief inverse for this economy. Determine whether the Leontief model presented in this problem is productive.

$$
P=
\begin{bmatrix}
    193  & 157 & 126 \\
    270 & 231  & 72 \\
    172 & 213 & 245  \\
\end{bmatrix}
$$

$$
b=
\begin{bmatrix}
    942 \\
    963 \\
    1115  \\
\end{bmatrix}
$$

In [14]:
P = np.array([[193, 157, 126], [270, 231, 72], [172, 213, 245]])
b = np.array([942, 963, 1115]).T

final_demand_vector = b - (P @ np.ones_like(b))
exp_coefficients = P / b
leontief_matrix = np.linalg.inv(np.eye(3) - exp_coefficients)
print(final_demand_vector)
print(exp_coefficients)
print(leontief_matrix)

[466 390 485]
[[0.2  0.16 0.11]
 [0.29 0.24 0.06]
 [0.18 0.22 0.22]]
[[1.45 0.38 0.24]
 [0.59 1.5  0.21]
 [0.51 0.52 1.4 ]]


In [15]:
eigenvalues, _ = np.linalg.eig(leontief_matrix)
eigenvalues

array([2.23, 1.06, 1.06])

**Answer (Task 7).** $\lambda_{max} = 2.23 > 1 \rightarrow \text{not productive}$.

$$
\text{FDV} =
\begin{bmatrix}
466 & 390 & 485
\end{bmatrix}
$$

$$
C=
\begin{bmatrix}
 0.2 & 0.16 & 0.11 \\ 
 0.29 & 0.24 & 0.06 \\
 0.18 & 0.22 & 0.22 \\
\end{bmatrix}
$$

$$
\text{Leontief Matrix} =
\begin{bmatrix}
 1.45 & 0.38 & 0.24 \\ 
 0.59 & 1.5 & 0.21 \\
 0.51 & 0.52 & 1.4 \\
\end{bmatrix}
$$

___
$\text{Task 8}.$ Find the value $f(A)$, $f(l) = e^{l + 1}$, where
$$
A=
\begin{bmatrix}
 7 & 8 & 3 \\ 
 -13 & -18 & -7 \\
 37 & 56 & 21 \\
\end{bmatrix}
$$

1. Find the eigenvalues and eigenvectors to check whether the matrix is diagonalizable (no, it's not).
2. Since it's not trivially diagonalizable, find the Jordan form $SJS^-1$.
3. Perform Jordan-Chevalley decomposition $J = D + N$, where $D$ is a diagonal matrix, $N$ is nilpotent. $D$ and $N$ commute.
4. Calculate $e^D$ for matrix $D$ then add $N$, $e^A = Se^JS^-1$. 

In [16]:
A = np.array([[7.0, 8.0, 3.0], [-13.0, -18.0, -7.0], [37.0, 56.0, 21.0]])
eigenvalues, eigenvectors = np.linalg.eig(A)
eigenvalues.real, eigenvectors.real

(array([6., 2., 2.]),
 array([[-0.18,  0.58,  0.58],
        [ 0.37, -0.58, -0.58],
        [-0.91,  0.58,  0.58]]))

In [17]:
S, J = sp.Matrix(A).jordan_form()
S_inv = np.linalg.inv(np.array(S).astype(float))
S, J, S_inv

(Matrix([
 [ 0.5, -1.5,  0.2],
 [-0.5,  1.0, -0.4],
 [ 0.5,    0,  1.0]]),
 Matrix([
 [2.0, 1.0,   0],
 [  0, 2.0,   0],
 [  0,   0, 6.0]]),
 array([[-20., -30.,  -8.],
        [ -6.,  -8.,  -2.],
        [ 10.,  15.,   5.]]))

In [18]:
D = np.array([[2.0, 0.0, 0.0], [0.0, 2.0, 0.0], [0.0, 0.0, 6.0]])
N = np.array([[0.0, 1.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
# Matrices D and N must commute!
print((D @ N == N @ D).all()) 

expJ = np.array([[np.e ** 2.0, 1.0, 0.0], [0.0, np.e ** 2.0, 0.0], [0.0, 0.0, np.e ** 6.0]])
S @ expJ @ S_inv

True


Matrix([
[  796.46853088654, 1184.11921218141,  395.039737393804],
[-1581.15894957522, -2364.8493682639, -791.079474787609],
[ 3957.39737393804, 5936.59606090707,  1986.58774306795]])

**Answer (Task 8.)** See the matrix above.

___
$\text{Task 9}.$ Solve the following linear programming problem and find the "shadow prices" that meet each constraint.

$$
24x_1 +102x_2 + 24x_3 \rightarrow \text{min}\\
\begin{cases}
-8x_1 + 8x_2 + 6x_3 \ge 3 \\
4x_1 + 5x_2 - 5x_3 \ge 2 \\
x_1 \ge 0; x_2 \ge 0; x_3 \ge 0 
\end{cases}
$$

1. Form the dual problem.

$$
3y_1 + 2y_2 \rightarrow \text{max}\\
\begin{cases}
-8y_1 + 4y_2 \le 24 \\
8y_1 + 5y_2 \le 102 \\
6x_1 - 5y_2 \le 24 \\
y_1 \ge 0; y_2 \ge 0 
\end{cases}
$$

2. Just plug it into the solver.

In [19]:
import scipy
from scipy.optimize import linprog

f = np.array([24, 102, 24])
A = np.array([[-8, 8, 6],[4, 5, -5]])
b = np.array([3, 2]).T

dual_f, dual_b, dual_A = -b, f, A.T
x_dual = scipy.optimize.linprog(dual_f, dual_A, dual_b, method = 'simplex')
x_dual

     con: array([], dtype=float64)
     fun: -40.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.,  0., 70.])
  status: 0
 success: True
       x: array([ 4., 14.])

**Answer (Task 9):** Variables in the dual problem are **4** and **14**.

___
$\text{Task 10}.$ Construct such a matrix norm on the $M_{2\times2}$-space so that the series of matrices $A^n$ converges. 

$$\mathbf{A} = \begin{bmatrix}
-0.8 & 6.0 \\
0.0 & 0.8 \\
\end{bmatrix}
$$

It can be shown that $\lim_{n \rightarrow \infty} A^{n} = 0_{2,2}.$ 
Let $A \in C_{n \times n}$ with spectral radius ρ(A). Then ρ(A) < 1 iff $\lim_{k \rightarrow \infty} A_k = 0$. 
On the other hand, if ρ(A) > 1, $\lim_{k \rightarrow \infty} ‖ A_{k} ‖ = \infty$ The statement holds for any choice of matrix norm on $C_{n \times n}$.
The proof of this theorem is omitted (can be found here https://www-users.math.umn.edu/~olver/num_/lni.pdf). According to it, we can choose an arbitrary norm and it'll still converge to $0$ as $A^n$ converges to $0$ as well. 

In [20]:
A = np.array([[-0.8, 6.0], [0.0, 0.8]])
np.linalg.eigvals(A) # spec rad = 0.8

array([-0.8,  0.8])

**Answer (Task 10):** an arbitrary matrix norm, e.g. $$||A||_F = (\sum_{i = 1}^m \sum_{j = 1}^n |a_{ij}|^2)^{\frac{1}{2}}$$ 